In [13]:
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp
import sys
import scipy.io
sys.path.append("..")
import ODE_Dynamics as od
import Test_Likelihood as tl
import Positive_Symptom_fn as fn

In [14]:
data = scipy.io.loadmat('../data/test_data.mat')
x = tf.cast(data['data'],dtype = tf.float32)
test_data = np.reshape(
      x, x.shape)

In [15]:
test_data = tf.transpose(test_data)

In [16]:
test_data[...,0]

<tf.Tensor: shape=(31,), dtype=float32, numpy=
array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30.], dtype=float32)>

In [17]:
vdyn_ode_fn = od.ViralDynamics

In [18]:
positive_fn = fn.proba_pos_sym(1.7).positive_fn
symptom_fn = fn.proba_pos_sym(1.7).symptom_fn

In [19]:
prob_s_ibar = 0.1

In [20]:
loglike = tl.loglik(test_data, vdyn_ode_fn, positive_fn, symptom_fn, prob_s_ibar, prob_fp=0.0, Epi_Model=od.SIR,
                 duration=24.0, Epi_cadence=0.5, Vir_cadence=0.0625)

In [21]:

# epipar = tf.constant([
#   [[2.2, 3.1E-05,0.8],
#    [2.4, 3.6E-05,0.9]],
#   [[1.8, 2.6E-05,0.83],
#    [1.7, 2.9E-05,0.88]],
#   [[2.0, 3.3E-05,0.78],
#    [2.2, 3.0E-05,0.75]],])

mu_b, sigma_b = 5, 1
beta = np.random.normal(mu_b, sigma_b, 1)
L = 0.0025/beta
V0 = np.random.normal(1E3, 1E2, 1)
print(V0)
X0 = 1E6
Y0 = V0
par=tf.constant(np.array([[L,0.01,beta*1E-7,0.5,20,10, V0, X0, Y0]], dtype=np.float32))

# par = tf.constant([
#   [[5E3,0.01,5E-7,0.5,20,10],
#    [5E3,0.01,5E-7,0.5,20,10]],
#   [[5E3,0.01,5E-7,0.5,20,10],
#    [5E3,0.01,5E-7,0.5,20,10]],
#   [[5E3,0.01,5E-7,0.5,20,10],
#    [5E3,0.01,5E-7,0.5,20,10]],])
vpar = par
pospar = par
sympar = par

[1045.21569763]


In [22]:
sample_size = 30
index = 1
mu_b, sigma_b = 5, 1
beta = np.random.normal(mu_b, sigma_b, 1)   #"rate at which virus infects host cells"
L = 0.0025/beta
par=np.array([[L,0.01,beta*1E-7,0.5,20.0,10.0]])
V0 = np.random.normal(1E3, 1E2, 1)
X0 = 1E6
Y0 = V0
init_state=(np.array([[V0,X0,Y0]], dtype=np.float32))

while index <= sample_size:
    beta = np.random.normal(mu_b, sigma_b, 1)   #"rate at which virus infects host cells"
    L = 0.0025/beta
    par_new=np.array([[L,0.01,beta*1E-7,0.5,20.0,10.0]])
    par = np.concatenate((par, par_new), axis = 0)
    V0 = np.random.normal(1E3, 1E2, 1)
    X0 = 1E6
    Y0 = V0
    init_state_new=(np.array([[V0,X0,Y0]], dtype=np.float32))
    init_state = np.concatenate((init_state, init_state_new), 0)
    
    index +=1

vpar = tf.constant(par, dtype=tf.float32)
pospar = par
sympar = par
print(vpar.shape)

(31, 6)


In [23]:
epipar = tf.constant(np.array([[1.8, 5E-8,0.1, 0.001,0.999]], dtype=np.float32))

In [24]:
ll = loglike.__call__(test_data,epipar, vpar, pospar, sympar)

printing solution times
tf.Tensor(
[0.     0.0625 0.125  0.1875 0.25   0.3125 0.375  0.4375 0.5    0.5625
 0.625  0.6875 0.75   0.8125 0.875  0.9375 1.     1.0625 1.125  1.1875
 1.25   1.3125 1.375  1.4375 1.5    1.5625 1.625  1.6875 1.75   1.8125
 1.875  1.9375 2.     2.0625 2.125  2.1875 2.25   2.3125 2.375  2.4375
 2.5    2.5625 2.625  2.6875 2.75   2.8125 2.875  2.9375 3.     3.0625
 3.125  3.1875 3.25   3.3125 3.375  3.4375 3.5    3.5625 3.625  3.6875
 3.75   3.8125 3.875  3.9375 4.     4.0625 4.125  4.1875 4.25   4.3125
 4.375  4.4375 4.5    4.5625 4.625  4.6875 4.75   4.8125 4.875  4.9375
 5.     5.0625 5.125  5.1875 5.25   5.3125 5.375  5.4375 5.5    5.5625
 5.625  5.6875 5.75   5.8125 5.875  5.9375 6.     6.0625 6.125  6.1875
 6.25   6.3125 6.375  6.4375 6.5    6.5625 6.625  6.6875 6.75   6.8125
 6.875  6.9375 7.     7.0625 7.125  7.1875 7.25   7.3125 7.375  7.4375
 7.5    7.5625 7.625  7.6875 7.75   7.8125 7.875  7.9375 8.     8.0625
 8.125  8.1875 8.25   8.3125 8.375  8.4375

In [13]:
ll

<tf.Tensor: shape=(), dtype=float32, numpy=-36086.438>